Übung : Container Life-Cycle
-----------------------------------------


### Auswahl der Image Registry (Dev + Ops)

Wählt eine Image Registry aus, in unserem Beispiel verwenden wir wieder https://hub.docker.com/.

Falls, noch nicht erfolgt, Erstellt einen Accounts auf https://hub.docker.com/.

Und dann:
* Username und Password als Umgebungsvariablen setzen (als Python Script)
* `docker login` ausführen

In [ ]:
import os
name = input("Username: ")
os.environ["USER"] = name

from getpass import getpass
password = getpass("Password: ")
os.environ["PW"] = password

In [ ]:
! docker login -u ${USER} -p ${PW}

- - -
### Erstellung des Images (Dev)

Dazu erstellen wir zuerst unseren (PHP) Microservice

In [ ]:
%%bash
cat <<%EOF% >index.php
 <?php echo '<p>Hallo Welt</p>'; ?>
%EOF%

Und für die Paketierung des Microservices, als Container Image, ein Dockerfile

In [ ]:
%%bash
cat <<%EOF% >Dockerfile
FROM php:7.0-apache  
COPY index.php /var/www/html/
%EOF%

Beides zusammen ergibt mit `docker build` unser Container Image.

In [ ]:
%%bash
docker build -t index.docker.io/${USER}/myservice .

Das Resultat ist ein neues Image und das, dass Base Image (php) heruntergeladen wurde:

In [ ]:
! docker image ls

- - - 
### Testen des Images (Dev)

Um das Image zu Testen führen wir es aus und kontrollieren die Ausgabe.

**Hinweis**: Statt eines manuellen Tests würde sich hier [Continuous Integration](https://de.wikipedia.org/wiki/Kontinuierliche_Integration) mit einem Unit Test anbieten.

In [ ]:
! docker run -d --rm --name myservice index.docker.io/${USER}/myservice 
! docker logs myservice

In [ ]:
! docker kill myservice

- - -
### Speichern des Images in der Registry (Dev)

Wenn wir sicher, sind das alles läuft Pushen wir das Container Image in unsere Registry.

In [ ]:
! docker push index.docker.io/${USER}/myservice

***
# Produktion (Ops)

Das Operation, holt das Container Image von der Registry und startet dieses mittels Kubernetes.

In [ ]:
! kubectl run myservice --image=${USER}/myservice --restart=Never
! kubectl expose pod/myservice --type="LoadBalancer" --port 80 
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service myservice -o=jsonpath='{ .spec.ports[0].nodePort }')/"

***
Aufräumen

In [ ]:
! kubectl delete pod/myservice
! kubectl delete service/myservice